In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D, Dense, Dropout, Flatten  
from keras.layers.normalization import BatchNormalization

In [3]:
train = pd.read_csv('../input/digit-recognizer/train.csv')
test = pd.read_csv('../input/digit-recognizer/test.csv')
subm = pd.read_csv('../input/digit-recognizer/sample_submission.csv')


In [4]:
X = train.drop(['label'], 1).values
y = train['label'].values
test_x = test.values
X=X/255.0
test_x=test_x/ 255.0

X = X.reshape(-1,28,28,1)
test_x = test_x.reshape(-1,28,28,1)


In [5]:
from keras.utils import to_categorical
y = to_categorical(y)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

In [7]:
model=Sequential()
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu", input_shape=(28,28,1)))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))
model.add(Dense(10,activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(featurewise_center=False, samplewise_center=False, featurewise_std_normalization=False, samplewise_std_normalization=False, 
                             zca_whitening=False, rotation_range=20, zoom_range = 0.1, width_shift_range=0.01, height_shift_range=0.05, horizontal_flip=False, 
                             vertical_flip=False)

In [9]:
train_ = generator.flow(X_train, y_train, batch_size=100)
test_ = generator.flow(X_test, y_test, batch_size=100)

In [10]:
steps=X_train.shape[0] // 100
v_steps=X_test.shape[0] // 100

In [11]:
H = model.fit_generator(train_, epochs = 50, steps_per_epoch = steps,validation_steps = v_steps,validation_data = test_)

Epoch 1/50
378/378 [==============================] - 12s 33ms/step - loss: 0.1457 - accuracy: 0.9544 - val_loss: 0.3269 - val_accuracy: 0.9198
Epoch 2/50
378/378 [==============================] - 12s 32ms/step - loss: 0.0615 - accuracy: 0.9807 - val_loss: 0.0636 - val_accuracy: 0.9810
Epoch 3/50
378/378 [==============================] - 14s 36ms/step - loss: 0.0459 - accuracy: 0.9866 - val_loss: 0.0425 - val_accuracy: 0.9869
Epoch 4/50
378/378 [==============================] - 12s 32ms/step - loss: 0.0438 - accuracy: 0.9867 - val_loss: 0.0813 - val_accuracy: 0.9764
Epoch 5/50
378/378 [==============================] - 12s 32ms/step - loss: 0.0383 - accuracy: 0.9876 - val_loss: 0.0618 - val_accuracy: 0.9831
Epoch 6/50
378/378 [==============================] - 12s 33ms/step - loss: 0.0349 - accuracy: 0.9893 - val_loss: 0.0340 - val_accuracy: 0.9874
Epoch 7/50
378/378 [==============================] - 13s 33ms/step - loss: 0.0317 - accuracy: 0.9906 - val_loss: 0.0461 - val_accuracy:

In [13]:
pred = model.predict_classes(test_x, verbose=1)
subm['Label'] = pred
subm.to_csv("CNN_subm.csv", index=False)

875/875 [==============================] - 1s 2ms/step


NameError: name 'subm' is not defined